In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", version="3.0.0")
print(f"Features: {dataset['train'].column_names}")

  0%|          | 0/3 [00:00<?, ?it/s]

Features: ['article', 'highlights', 'id']


In [5]:
#article, which contains the news articles, highlights with the summaries, and id to uniquely identify each article
sample = dataset["train"][1]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}):
""")
print(sample["article"][:400])
print(f'\nSummary (length: {len(sample["highlights"])}):')
print(sample["highlights"])


Article (excerpt of 500 characters, total length: 4051):

Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the M

Summary (length: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


In [6]:
sample_text = dataset["train"][1]["article"][:2000]
# We'll collect the generated summaries of each model in a dictionary
summaries = {}

In [7]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

['The U.S. are a country.', 'The U.N. is an organization.']

In [9]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

In [10]:
summaries["baseline"] = three_sentence_summary(sample_text)

In [11]:

from transformers import pipeline, set_seed



In [12]:
#T5
pipe_t5 = pipeline("summarization", model="t5-large")
pipe_out_t5 = pipe_t5(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out_t5[0]["summary_text"]))

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [13]:
#BART
pipe_bart = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out_bart = pipe_bart(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out_bart[0]["summary_text"]))

In [14]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
#PEGASUS
pipe_pega = pipeline("summarization", model="google/pegasus-cnn_dailymail")
pipe_out_pega = pipe_pega(sample_text)
summaries["pegasus"] = pipe_out_pega[0]["summary_text"].replace(" .<n>", ".\n")

In [16]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

GROUND TRUTH
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .

BASELINE
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."

T5
mentally ill inmates are housed on the ninth floor of a florida jail .
most face drug charges or charges of assaulting an officer .
judge says arrests often result from confrontations with police .
one-third of all peopl

Pegasus

In [2]:
# Importing dependencies from transformers
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [3]:
# Load tokenizer 
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

In [4]:
# Load model 
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

In [77]:
text = """Gladly is a customer service platform for digitally-focused B2C companies who want to maximize the lifetime value of their customers.   Unlike the legacy approach to customer service software, which is designed around a ticket or case to enable workflows, Gladly enables radically personal customer service centered around people to sustain customer loyalty and drive more revenue.

The world’s most innovative consumer companies like Godiva, JOANN, and TUMI use Gladly to create lasting customer relationships, not one-off experiences. ."""

In [78]:
# Create tokens - number representation of our text
tokens = tokenizer(text, return_tensors="pt")

In [79]:
tokens

{'input_ids': tensor([[    0,   534, 15356,   352,    16,    10,  2111,   544,  1761,    13,
         25318,    12, 12804,   163,   176,   347,   451,    54,   236,     7,
         16189,     5,  7370,   923,     9,    49,   916,     4,  1437,  1437,
          8280,     5,  5184,  1548,     7,  2111,   544,  2257,     6,    61,
            16,  1887,   198,    10,  3682,    50,   403,     7,  3155,   173,
         17718,     6, 15151,   352,  9849, 26396,  1081,  2111,   544, 14889,
           198,    82,     7,  9844,  2111, 10177,     8,  1305,    55,   903,
             4, 50118, 50118,   133,   232,    17,    27,    29,   144,  5497,
          2267,   451,   101,  1840,  7222,     6, 30314, 15118,     6,     8,
           255,  5725,   100,   304, 15151,   352,     7,  1045,  9735,  2111,
          4158,     6,    45,    65,    12,  1529,  3734,     4,   479,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 

In [80]:
# Summarize 
summary = model.generate(tokens['input_ids'], early_stopping = False)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 142 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [81]:
summary

tensor([[    2,     0,   534, 15356,   352,    16,    10,  2111,   544,  1761,
            13, 25318,    12, 12804,   163,   176,   347,   451,     4,  8280,
             5,  5184,  1548,     7,  2111,   544,  2257,     6,    61,    16,
          1887,   198,    10,  3682,    50,   403,     7,  3155,   173, 17718,
             6, 15151,   352,  9849, 26396,  1081,  2111,   544, 14889,   198,
            82,     4,  1840,  7222,     6, 30314, 15118,     6,     8,   255,
          5725,   100,   304, 15151,   352,     7,  1045,  9735,  2111,  4158,
             6,    45,    65,    12,  1529,  3734,     4,     2]])

In [82]:
# Decode summary
print([tokenizer.decode(g, skip_special_tokens=True) for g in summary])

['Gladly is a customer service platform for digitally-focused B2C companies. Unlike the legacy approach to customer service software, which is designed around a ticket or case to enable workflows, Gladly enables radically personal customer service centered around people. Godiva, JOANN, and TUMI use Gladly to create lasting customer relationships, not one-off experiences.']


In [83]:
print(text)

Gladly is a customer service platform for digitally-focused B2C companies who want to maximize the lifetime value of their customers.   Unlike the legacy approach to customer service software, which is designed around a ticket or case to enable workflows, Gladly enables radically personal customer service centered around people to sustain customer loyalty and drive more revenue.

The world’s most innovative consumer companies like Godiva, JOANN, and TUMI use Gladly to create lasting customer relationships, not one-off experiences. .


In [84]:
def text_summarization_pegasus(text):
  tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
  model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
  inputs = tokenizer([text],return_tensors="pt")
  summary_ids = model.generate(inputs['input_ids'], early_stopping= False)
  return [tokenizer.decode(k, skip_special_tokens=True) for k in summary_ids]


In [85]:
text1 = """The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."""

In [86]:
text_summarization_pegasus(text1)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 64 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


['The Eiffel Tower is a landmark in Paris, France.']

# BART

In [39]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

In [88]:
ARTICLE_TO_SUMMARIZE = "The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."


In [89]:
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], return_tensors='pt')

In [90]:

summary_ids = model.generate(inputs['input_ids'],early_stopping=False)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 142 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [91]:
summary_ids

tensor([[    2,     0,   133,  9368,    16, 36593,  7472,    36,   134,     6,
          4124,   246, 16935,    43,  6764,     6,    59,     5,   276,  6958,
            25,    41,  7694,    12,  8005,   219,   745,     4,  3139,  1542,
            16,  3925,     6, 14978, 10529,  7472,    36, 31132, 16935,    43,
            15,   349,   526,     4,  3015, 19586, 25556,  2696,     6,     5,
           381,  4822,   523,  7186,    16,     5,   200, 28038,   481,    12,
          8190,  3184,    11,  1470,    71,     5,  5388,  1180, 16376,   625,
         21491,     4,     2]])

In [92]:
print([tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids])

['The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building. Its base is square, measuring 125 metres (410 ft) on each side. Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct.']


In [93]:
print(ARTICLE_TO_SUMMARIZE )

The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct.


In [94]:
def text_summarization_bart(text):
  model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
  tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
  inputs = tokenizer([text], return_tensors='pt')
  summary_ids = model.generate(inputs['input_ids'],early_stopping=False)
  return [tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids]

In [95]:
text2 = """Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.[32]

Python is dynamically-typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.[33][34]

Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0.[35] Python 2.0 was released in 2000 and introduced new features such as list comprehensions, cycle-detecting garbage collection, reference counting, and Unicode support. Python 3.0, released in 2008, was a major revision that is not completely backward-compatible with earlier versions. Python 2 was discontinued with version 2.7.18 in 2020.[36]"""

In [96]:
text_summarization_bart(text2)

['Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000 and introduced new features such as list comprehensions, cycle-detecting garbage collection, reference counting, and Unicode support. Python 3.0, released in 2008, was a major revision that is not completely backward-compatible with']

In [98]:
print(text2)

Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.[32]

Python is dynamically-typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.[33][34]

Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0.[35] Python 2.0 was released in 2000 and introduced new features such as list comprehensions, cycle-detecting garbage collection, reference counting, and Unicode support. Python 3.0, released in 2008, was a major revision that is not completely backward-compatible with earlier versions. Python 2 was discontinued with version 2.7.18 in 2020.[36]
